In [6]:
import pandas as pd
import os
import re

def split_by_cancer_type_wide(
    excel_path: str,
    output_dir: str = "cancer_grouped_wide_csvs",
):
    df = pd.read_excel(excel_path)

    # --- Find the key header rows by searching column 1 (the "label" column) ---
    # This makes it robust even if row numbers shift slightly.
    label_col = df.iloc[:, 1].astype(str)

    def find_row_index(needle: str) -> int:
        hits = df.index[label_col.str.strip() == needle]
        if len(hits) == 0:
            raise ValueError(f"Couldn't find row labeled '{needle}' in column 2.")
        return int(hits[0])

    cancer_row = find_row_index("Cancer Type")
    cellline_row = find_row_index(r"Gene\Cell-Line Name")  # matches exactly in your file

    # Genes start on the next row after "Gene\Cell-Line Name"
    genes_start_row = cellline_row + 1

    # --- Identify the "cell line" data columns (start from column index 2 onward) ---
    data_cols = list(df.columns[2:])

    # Cancer type label per column (e.g., PRAD, BRCA, "KICH, KIRC, KIRP", etc.)
    cancer_labels = df.loc[cancer_row, data_cols]

    # Cell line name per column (e.g., 22RV1)
    cellline_names = df.loc[cellline_row, data_cols]

    # Gene names column
    gene_series = df.loc[genes_start_row:, df.columns[1]].dropna().astype(str)

    # Values matrix: rows=genes, cols=cell lines
    values = df.loc[genes_start_row:genes_start_row + len(gene_series) - 1, data_cols]

    # --- Build mapping cancer_type -> list of columns belonging to it ---
    cancer_to_cols = {}
    for col in data_cols:
        clabel = cancer_labels[col]
        if pd.isna(clabel):
            continue

        # split combined labels like "KICH, KIRC, KIRP"
        cancers = [c.strip() for c in str(clabel).split(",") if c.strip()]
        for c in cancers:
            cancer_to_cols.setdefault(c, []).append(col)

    # --- Write one CSV per cancer type, preserving the 0/1 matrix ---
    os.makedirs(output_dir, exist_ok=True)

    saved = []
    for cancer, cols in cancer_to_cols.items():
        sub = values[cols].copy()

        # Rename columns to cell line names (fallback to original column if missing)
        rename_map = {}
        for col in cols:
            name = cellline_names[col]
            if pd.isna(name) or str(name).strip() == "":
                rename_map[col] = str(col)
            else:
                rename_map[col] = str(name).strip()
        sub = sub.rename(columns=rename_map)

        # Insert gene column as first column
        out_df = pd.DataFrame({"gene": gene_series.values})
        out_df = pd.concat([out_df, sub.reset_index(drop=True)], axis=1)

        # Sanitize filename
        safe_cancer = re.sub(r"[^A-Za-z0-9_\-]+", "_", cancer)
        out_path = os.path.join(output_dir, f"{safe_cancer}.csv")
        out_df.to_csv(out_path, index=False)
        saved.append(out_path)

    print(f"Saved {len(saved)} files to: {output_dir}")
    print("Example:", saved[:5])

# ---- run it ----
split_by_cancer_type_wide("Data_Edited.xlsx")


Saved 24 files to: cancer_grouped_wide_csvs
Example: ['cancer_grouped_wide_csvs/PRAD.csv', 'cancer_grouped_wide_csvs/STAD.csv', 'cancer_grouped_wide_csvs/GBM.csv', 'cancer_grouped_wide_csvs/KICH.csv', 'cancer_grouped_wide_csvs/KIRC.csv']


In [4]:
expanded_records = []

for _, row in long_df.iterrows():
    cancers = [c.strip() for c in row["cancer_type"].split(",")]
    for c in cancers:
        expanded_records.append({
            "gene": row["gene"],
            "cancer_type": c
        })

expanded_df = pd.DataFrame(expanded_records).drop_duplicates()
print(expanded_df.head())

      gene cancer_type
0     A1BG        PRAD
1     A1CF        PRAD
2     AAAS        PRAD
3     AACS        PRAD
4  AADACL2        PRAD


In [9]:
import pandas as pd
import os
import re

def split_by_cancer_type_with_headers(
    excel_path: str,
    output_dir: str = "cancer_grouped_excels_with_headers",
):
    df = pd.read_excel(excel_path)

    # Find key rows dynamically
    label_col = df.iloc[:, 1].astype(str)

    def find_row(label):
        return df.index[label_col.str.strip() == label][0]

    tissue_row = find_row("Tissue")
    cancer_row = find_row("Cancer Type")
    cmp_row = find_row("CMP id")
    cellline_row = find_row("Gene\\Cell-Line Name")

    genes_start = cellline_row + 1

    data_cols = list(df.columns[2:])

    tissue_labels = df.loc[tissue_row, data_cols]
    cancer_labels = df.loc[cancer_row, data_cols]
    cmp_ids = df.loc[cmp_row, data_cols]
    cellline_names = df.loc[cellline_row, data_cols]

    gene_names = df.loc[genes_start:, df.columns[1]].dropna().astype(str)
    matrix = df.loc[genes_start:genes_start + len(gene_names)-1, data_cols]

    # Map cancer → columns
    cancer_to_cols = {}
    for col in data_cols:
        label = cancer_labels[col]
        if pd.isna(label):
            continue
        cancers = [c.strip() for c in str(label).split(",")]
        for c in cancers:
            cancer_to_cols.setdefault(c, []).append(col)

    os.makedirs(output_dir, exist_ok=True)

    for cancer, cols in cancer_to_cols.items():
        # --- HEADER BLOCK ---
        header_df = pd.DataFrame({
            "Label": ["Tissue", "Cancer Type", "CMP id", "Gene\\Cell-Line Name"]
        })

        for col in cols:
            header_df[cellline_names[col]] = [
                tissue_labels[col],
                cancer_labels[col],
                cmp_ids[col],
                cellline_names[col]
            ]

        # --- GENE MATRIX ---
        gene_df = pd.DataFrame({"Gene": gene_names.values})
        gene_matrix = matrix[cols].reset_index(drop=True)
        gene_matrix.columns = [cellline_names[c] for c in cols]

        gene_df = pd.concat([gene_df, gene_matrix], axis=1)

        # Combine header + genes
        final_df = pd.concat([header_df, gene_df], ignore_index=True)

        # Save Excel
        safe_name = re.sub(r"[^A-Za-z0-9_\-]+", "_", cancer)
        out_path = f"{output_dir}/{safe_name}.xlsx"
        final_df.to_excel(out_path, index=False, header=False)

        print("Saved:", out_path)

# Run
split_by_cancer_type_with_headers("Data_Edited.xlsx")


Saved: cancer_grouped_excels_with_headers/PRAD.xlsx
Saved: cancer_grouped_excels_with_headers/STAD.xlsx
Saved: cancer_grouped_excels_with_headers/GBM.xlsx
Saved: cancer_grouped_excels_with_headers/KICH.xlsx
Saved: cancer_grouped_excels_with_headers/KIRC.xlsx
Saved: cancer_grouped_excels_with_headers/KIRP.xlsx
Saved: cancer_grouped_excels_with_headers/SKCM.xlsx
Saved: cancer_grouped_excels_with_headers/HNSC.xlsx
Saved: cancer_grouped_excels_with_headers/OV.xlsx
Saved: cancer_grouped_excels_with_headers/LUAD.xlsx
Saved: cancer_grouped_excels_with_headers/MISC.xlsx
Saved: cancer_grouped_excels_with_headers/PAAD.xlsx
Saved: cancer_grouped_excels_with_headers/BRCA.xlsx
Saved: cancer_grouped_excels_with_headers/LGG.xlsx
Saved: cancer_grouped_excels_with_headers/COAD.xlsx
Saved: cancer_grouped_excels_with_headers/READ.xlsx
Saved: cancer_grouped_excels_with_headers/SARC.xlsx
Saved: cancer_grouped_excels_with_headers/ESCA.xlsx
Saved: cancer_grouped_excels_with_headers/UCEC.xlsx
Saved: cancer_gr